# HOMEWORK 3

## 1) DATA COLLECTION

### 1.1 Get the list of animes
We start from the list of animes to include in your corpus of documents. In particular, we focus on the top animes ever list. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

The output of this step is a .txt file whose single line corresponds to an anime's url.

## Solution:
The main idea is to split the work between the retrieving of the html pages (skipping the already stored urls) and the retrieving of the links in the pages.<br>
Once we've defined a way to scrap the urls from the page, we'll iterate over 20000 to get all the pages till that number. At the time of execution the urls retrieved was only 19122, but in other execution it was even 19128, so it depends from the moment you execute that. The file was created in 5-Nov-2021 so the list refers to the content of the site in that day.

In [2]:
#import libraries
import requests
url = 'https://myanimelist.net/topanime.php'
response= requests.get(url)
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
def get_links_from_soup(soup):
    ''' 
        Given a BeautifulSoup object the functions iterates over all the links in the table rows in the corrisponding page.
        The functions also returns the list containing all the destination (href) of these links
    '''
    anime = []
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:
            # checking if there is a content in the link
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')))
    return anime

In [ ]:
tot_list= []
for lim in range(0, 20000, 50):
    
    # The principal page of animelis, so: https://myanimelist.net/topanime.php
    if lim==0:
        new_url = url
    else: # we've to skio the first lim elements
        new_url = url+'?limit='+str(lim)
    response = requests.get(new_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tot_list += get_links_from_soup(soup)

In [ ]:
len(tot_list)

19122

In [ ]:
# Given that the list contains couples in the form: name, link and we want only the link, we save only the second element

with open('out.txt', 'w') as f:
    for n, link in tot_list:
        f.write(link+'\n')

### 1.2. Crawl animes
Once you get all the urls in the first 400 pages of the list, you:

* Download the html corresponding to each of the collected urls.
* After you collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the * * * stopping point. More details in Important (2).
* Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

**Important**

Due to the large amount of pages you need to download, follow the next tips that will help you speeding up several time-consuming operations.

[Save time downloading files] You are asked to crawl a considerable number of pages, which will take plenty of time. To speed up the operation, we suggest you to work in parallel with your group's colleagues or even generate code that works in parallel with all the CPUs available in your computer. In particular, using the same code, each component of the group can be in charge of downloading a subset of pages (e.g., the first 100). PAY ATTENTION: Once obtained all the pages, merge your results into an unique dataset. In fact, the search engine must look up for results in the whole set of documents.

[Save your data] It is not nice to restart a crawling procedure, given its runtime. For this reason, it is extremely important that for every time you crawl a page, you must save it with the name article_i.html, where i corresponds to the number of articles you have already downloaded. In such way, if something goes bad, you can restart your crawling procedure from the i+1-th document.

## Solution:
Once the website has to protect itself, it will block our requests if they are too much, so i've looked first if there's an error and, in that case, the program stops for a random range of time.<br>
In this way we've collected only cleaned pages (not the one containing the error button).<br>
The program (that can be found in the [downloder.py](downloader.py) file) has not been run by this notebook, but we've made a CLI version in order to run it from the terminal in how many terminal you want in order to work in parallel.<br>
The specification of that function can be retrieved in that file and even launching it from terminal with the ```--h``` flag only to see the documentation and the usage.

### 1.3 Parse downloaded pages
At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

* **Anime Name** (to save as animeTitle): *String*
* **Anime Type** (to save as animeType): *String*
* **Number of episode** (to save as animeNumEpisode): *Integer*
* **Release and End Dates of anime** (to save as releaseDate and endDate): Convert both release and end date into *datetime format*.
* **Number of members** (to save as animeNumMembers): *Integer*
* **Score** (to save as animeScore): *Float*
* **Users** (to save as animeUsers): *Integer*
* **Rank** (to save as animeRank): *Integer*
* **Popularity** (to save as animePopularity): *Integer*
* **Synopsis** (to save as animeDescription): *String*
* **Related Anime** (to save as animeRelated): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. *List of strings*.
* **Characters** (to save as animeCharacters): *List of strings*.
* **Voices** (to save as animeVoices): *List of strings*
* **Staff** (to save as animeStaff): Include the staff name and their responsibility/task in a *list of lists*.

In [5]:
import html_parser

## Solution:

All the needed functions are in the [html_parser.py](./html_parser.py) file that contains the relative comments and documentation too.
The only function that has to be called to entirely solve this point is the ```save_tsv_info``` function.<br>
Even if the function that will be called in order to get all the needed data in the tsv format is only one, the function implemented are several and more or less they are grouped in three category:
* support function (to obtain a BeautifulSoup object from the html o to retrieve the datetime format from a string)
* webscraping function: They are the core of this implementation, all of them takes as parameter a BeautifulSoup object and a dictionary and their purpouse is to put into this dictionary as much information as possible starting from the object in input
* applicative function: They allow to directly obtain the data from a file o an index employing the other functions (i.e. ```save_tsv_info```)

### Webscraping:
In order to retrieve the 13 required fields we need first of all to study the structure of the page.<br>
In general the title is simple to retrieve using bs.<br>
The page, in general is structured as a series of nested tables, so we have a huge table with two columns, one for the left menu that contains all the info relatives to the score, the type of the serie, the ranking and so on and the other is the real body of the page, containing the synopsis, the related animes and other required fields.<br>
<newline>
Considering that, we have written a function called ```get_left_attributes``` that retrieves the values for the fields:
* episodes
* start_date
* end_date
* score
* users
* rank
* members
* popularity
* type
These fields are all in one div for each one and this div has the class equal to '*spaceit_pad*', then we have as content of this div a list of values in the form (at exception of the score) ```['\n', name_of_the_value, value]``` then value has to be processed in order to extract the date or the integer and so on.<br>
<newline>

The remaining values can be found in the right column of the huge table inside different divs or subtables.<br>
<newline>

#### Synopsis<br>
Can be found simply in a paragraph with the attribute '*itemprop*' equal to 'description'<br>
#### Related animes<br>
They are inside a dedicated table with class '*anime_detail_related_anime*', in particular we want only the animes for which exists a link, so we'll iterate over all the links in that table and we'll take the different contents returning the unique values
#### Staff<br>
In this case (as for the next) the function will work directly on a specific div (the divs that has the class '*detail-characters-list clearfix*')<br>
Given that div we have all the names that we need as content of links so we'll simply iterate over it and collect the data of interest
#### Characters and voices<br>
This div has the same class of the above one, so if we have only one of the two we can distinguish them by the content of an h3 element of the class '*h3_characters_voice_actors*'.<br>
Starting from this div we have other two div in order to split the list in two columns, these divs contains a list of tables, each of them contains only one row and three columns, one for the images of the characters, one for the name of the characters and the third for the voice, so we'll iterate over the row elements and selecting only the second two columns.<br>
From the first and the second we'll get the link element containing the name either of the character or for the voice.
#### Putting all togheter<br>
It's done with a function that uses each of these subfunction in order to retrieve, given an html page, the content of interest in a dictionary
<newline>

Once we can obtain a dictionary with the data of interest converting and saving it in a tsv file is trivial and it's done by ad-hoc functions. All this work is encapsulated in the funciton ```save_tsv_info``` that takes the ranges of the page for which we that we want to save the scraped tsv, the base directory where these page are stored and the destination directory over which the outputs will be stored.

In [6]:
html_parser.get_total_info('../data/html_pages/article_00000.html')

{'title': 'Fullmetal Alchemist: Brotherhood',
 'type': 'TV',
 'episodes': 64,
 'start_date': datetime.datetime(2009, 4, 5, 0, 0),
 'end_date': datetime.datetime(2010, 7, 4, 0, 0),
 'score': 9.16,
 'users': 1622384,
 'ranked': 1,
 'popularity': 3,
 'members': 2675906,
 'synopsis': "After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.",
 'related_anime': ['Fullmetal Alchemist: Brotherhood - 4-Koma Theater',
  'Fullmetal Alchemist: Brotherhood Specials',
  'Fullmetal Alchemist',
  'Fullmetal Alchemist: The Sacred Star of Milos'],
 'characters': ['Elric, Edward

For each anime, you create an anime_i.tsv file of this structure:

animeTitle \t animeType \t  ... \t animeStaff

## Solution:
In this case we've simply runned the function ```save_tsv_info``` starting from 0 to 19123 (the total number of the pages) in order to get all the necessary.
An example of the output of the funcion that gets the tsv is the following:

In [7]:
header, content = html_parser.get_tsv_from_idx(0)
print(f"The header of the tsv table is the following:\n\t{header}")
print()
print(f"The content for the 0-th page (the one of the dictionary above) is the following:\n\t{content}")

The header of the tsv table is the following:
	title	type	episodes	start_date	end_date	score	users	ranked	popularity	members	synopsis	related_anime	characters	voices	staff

The content for the 0-th page (the one of the dictionary above) is the following:
	Fullmetal Alchemist: Brotherhood	TV	64	2009-04-05 00:00:00	2010-07-04 00:00:00	9.16	1622384	1	3	2675906	After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.	['Fullmetal Alchemist: Brotherhood - 4-Koma Theater', 'Fullmetal Alchemist: Brotherhood Specials', 'Fullmetal Alchemist', 'Fullmetal Alchemist: The Sa

## Getting the TSVs!!!

In [8]:
# The following line is commented for safety reason, the execution will bring to an error given that the data doesn't exist in your machine

#html_parser.save_tsv_info(0, 1923) # We've used the default directory for source and dest, see the code for details

## 2. SEARCH ENGINE
Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.

First, you must pre-process all the information collected for each anime by:

* Removing stopwords
* Removing punctuation
* Stemming
* Anything else you think it's needed

For this purpose, you can use the nltk library.

### 2.1. Conjunctive query
For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

### 2.1.1) Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary of this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
where document_i is the id of a document that contains the word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

### 2.1.2) Execute the query
Given a query, that you let the user enter:

saiyan race
the Search Engine is supposed to return a list of documents.

What documents do we want?
Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

animeTitle
animeDescription
Url
Example Output:

animeTitle	animeDescription	Url
Fullmetal Alchemist: Brotherhood	...	https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood
Gintama	...	https://myanimelist.net/anime/28977/Gintama%C2%B0
Shingeki no Kyojin Season 3 Part 2	...	https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2
If everything works well in this step, you can go to the next point, and make your Search Engine more complex and better in answering queries.

### 2.2) Conjunctive query & Ranking score
For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

* Find all the documents that contains all the words in the query.
* Sort them by their similarity with the query.
* Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use é * Python libraries) for maintaining the top-k documents.

To solve this task, you will have to use the tfIdf score, and the Cosine similarity. The field to consider it is still the synopsis. Let's see how.

### 2.2.1) Inverted index

### 2.2.2) Execute the query

## 3. DEFINE A NEW SCORE!

## 5. Algorithmic question
You consult for a personal trainer who has a back-to-back sequence of requests for appointments. A sequence of requests is of the form > 30, 40, 25, 50, 30, 20 where each number is the time that the person who makes the appointment wants to spend. You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, [30, 50, 20] is an acceptable solution (of duration 100), but [30, 40, 50, 20] is not, because 30 and 40 are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is [40, 50, 20], of total duration 110.

* Write an algorithm that computes the acceptable solution with the longest possible duration.
* Implement a program that given in input an instance in the form given above, gives the optimal solution.

In [2]:
def algoritmo (array):
    a=0
    b=array[0]
    for elem in array[1:]:
        n=max(a+elem,b)
        a=b
        b=n
    return b

In [3]:
array=[3,4,5,60,4]
algoritmo(array)

64

In [4]:
caso_limite = [50, 65, 90, 60, 35, 10, 15, 25]
print(algoritmo(caso_limite))

200


In [14]:
def recursive_algo(arr):
    if len(arr)<2:
        return arr

    return(max(recursive_algo(arr[1:]), [arr[0]] + recursive_algo(arr[2:]), key=sum))

recursive_algo(caso_limite)

[50, 90, 35, 25]

In [5]:
def algoritmo_leo(array):
    a=0
    b=array[0]
    step=[array[0]]
    for elem in array[1:]:
        n=max(a+elem,b)
        if a+elem>b:
            step.append(elem)
        print(elem,'massimo fra',a+elem,b,': n=',n,'a=',a,'b=',b)
        a=b
        b=n
    return b,step

print(algoritmo_leo(caso_limite)[1])

65 massimo fra 65 50 : n= 65 a= 0 b= 50
90 massimo fra 140 65 : n= 140 a= 50 b= 65
60 massimo fra 125 140 : n= 140 a= 65 b= 140
35 massimo fra 175 140 : n= 175 a= 140 b= 140
10 massimo fra 150 175 : n= 175 a= 140 b= 175
15 massimo fra 190 175 : n= 190 a= 175 b= 175
25 massimo fra 200 190 : n= 200 a= 175 b= 190
[50, 65, 90, 35, 15, 25]
